In [1]:
import pandas as pd
import numpy as np

BAI 1

In [5]:
file_path = 'lab7.csv'
df = pd.read_csv(file_path)

print(df.head())

df.to_excel('orders.xlsx', index=False)

selected_columns = ['order_id', 'customer_id', 'total_price']
df_selected = df[selected_columns]

   order_id  customer_id  product_id  quantity  total_price
0         1           70          25         8       887.63
1         2           67          35        10        34.57
2         3            3          41         5       960.27
3         4           90          50         3       440.45
4         5           70          40         7       900.92


In [6]:
df_selected['store_id'] = np.random.randint(1,11,1000)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_5628\4192329138.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['store_id'] = np.random.randint(1,11,1000)


In [9]:
df_selected.to_csv('lab7_store_id.csv', index=False)

BAI 2

In [14]:
import numpy as np
from faker import Faker

In [15]:
fake = Faker()

store_data = pd.DataFrame({
    'store_id': range(1, 11),
    'store_name': [fake.company() for _ in range(1, 11)]
})

store_data

,store_id,store_name
0,1,"Weeks, Fischer and Green"
1,2,Burns and Sons
2,3,"Merritt, Morgan and Walker"
3,4,"Grant, Smith and Velazquez"
4,5,Green Group
5,6,"Hancock, Greene and Davis"
6,7,Morgan Group
7,8,Miller-Roach
8,9,Campbell LLC
9,10,Mathews-Reynolds


In [16]:
store_data.to_csv('store_id.csv', index=False)

In [17]:
print(df.info(),'\n')
print(df.describe())

print(df.sample(5))

with pd.HDFStore('orders.h5') as store:
    store.put('orders', df, format='table')

# Đọc dữ liệu từ file HDF5
df_hdf5 = pd.read_hdf('orders.h5', 'orders')
print(df_hdf5.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   order_id     1000 non-null   int64  
 1   customer_id  1000 non-null   int64  
 2   product_id   1000 non-null   int64  
 3   quantity     1000 non-null   int64  
 4   total_price  1000 non-null   float64
dtypes: float64(1), int64(4)
memory usage: 39.2 KB
None 

          order_id  customer_id   product_id     quantity  total_price
count  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000
mean    500.500000    49.544000    25.793000     5.513000   499.329280
std     288.819436    29.142672    14.504914     2.925473   282.995611
min       1.000000     1.000000     1.000000     1.000000    12.400000
25%     250.750000    24.000000    13.000000     3.000000   270.177500
50%     500.500000    50.000000    26.000000     5.000000   498.740000
75%     750.250000    75.000000    38.000000  

BAI 3

In [18]:
lab7_df = pd.read_csv('lab7_store_id.csv')
store_df = pd.read_csv('store_id.csv')

In [19]:
merged_df = lab7_df.merge(store_df, on='store_id', how='left')

merged_df.to_csv('lab7_merged.csv', index=False)

In [28]:
def process_chunk(chunk):
    summary = chunk.groupby('store_id').agg({'order_id': 'count', 'total_price': 'sum'})
    return summary

chunk_size = 1000
total_orders = 0
total_revenue = 0

for chunk in pd.read_csv('lab7_merged.csv', chunksize=chunk_size):
    chunk_summary = process_chunk(chunk)  # Kiểm tra kiểu dữ liệu của chunk_summary
    if isinstance(chunk_summary, pd.DataFrame):  
        total_orders += chunk_summary['order_id'].sum()
        total_revenue += chunk_summary['total_price'].sum()
    else:
        print("Warning: process_chunk() không trả về DataFrame, hãy kiểm tra lại.")

print(f'Tổng số lượng đơn hàng: {total_orders}')
print(f'Tổng doanh thu:{total_revenue}')

Tổng số lượng đơn hàng: 1000
Tổng doanh thu:499329.27999999997


BAI 4

In [30]:
store_summary = merged_df.groupby('store_id').agg({'order_id': 'count', 'total_price': 'sum'})
print(store_summary)

# Tính số lượng khách hàng đặt hàng từ mỗi cửa hàng
customer_count = merged_df.groupby('store_id')['customer_id'].nunique()
print(customer_count)

# Tính tổng số lượng đơn hàng và tổng doanh thu từ mỗi khách hàng
customer_summary = merged_df.groupby('customer_id').agg({'order_id': 'count', 'total_price': 'sum'})
print(customer_summary)

          order_id  total_price
store_id                       
1              107     47478.48
2              106     50981.38
3              105     50394.89
4              102     53070.54
5               97     49324.60
6              109     60376.92
7               99     50751.43
8               82     36494.27
9               89     45878.76
10             104     54578.01
store_id
1     67
2     66
3     62
4     63
5     64
6     64
7     64
8     55
9     57
10    64
Name: customer_id, dtype: int64
             order_id  total_price
customer_id                       
1                  13      6256.85
2                  10      4767.06
3                  11      4303.60
4                  17      9488.57
5                   9      5248.37
...               ...          ...
96                 11      4023.31
97                  5      3784.12
98                  6      3506.17
99                 12      3747.18
100                13      7111.60

[100 rows x 2 columns]
